### Расстояние Левенштейна

<a href="https://en.wikipedia.org/wiki/Levenshtein_distance">Расстояние Левенштейна</a> (Levenshtein distance) &mdash; это метрика, которая показывает, какое количество минимальных операций (замен, удалений, вставок) нужно совершить, чтобы из одной последовательности получить другую. Например, расстояние между строками "стул" и "стала" равно 2.

Расстояние Левенштейна &mdash; не единственный способ вычислить расстояние между двумя строками. Например, расстояние Хэмминга (которое было на втором курсе) вычисляется для строк равной длины и равно количеству позиций, на которых символы не совпадают.

Расстояние Левенштейна определяется следующим образом: пусть у нас есть две строки $a$ и $b$. Пусть $\text{head}(x)$ &mdash; первый символ строки, $\text{tail}(x)$ &mdash; все остальные. Тогда:


$$
\text{lev}(a, b) = \begin{cases}
|a| & \text{если } |b| = 0, \\
|b| & \text{если } |a| = 0, \\
\text{lev}(\text{tail}(a), \text{tail}(b)) & \text{если } \text{head}(a) = \text{head}(b), \\
1 + \min \begin{cases}
\text{lev}(\text{tail}(a), b) \\
\text{lev}(a, \text{tail}(b)) \\
\text{lev}(\text{tail}(a), \text{tail}(b))
\end{cases} & \text{в противном случае } \\
\end{cases}
$$

Расстояние Левенштейна можно вычислять рекурсивно, но это достаточно неэффективно. Лучше делать это итеративно, сохраняя расстояния между подстроками в специальную матрицу (алгоритм Вагнера&ndash;Фишера):

|||с|т|а|л|а|
|---|---|---|---|---|---|---|
||0|1|2|3|4|5|
|**с**|1|0|1|2|3|4|
|**т**|2|1|0|1|2|3|
|**о**|3|2|1|1|2|3|
|**л**|4|3|2|2|1|2|


Каждая ячейка в таблице соответствует расстоянию Левенштейна между подстроками: 
$$
\text{D}(i, j) = \text{lev}(a[1...i], b[1...j])
$$

Интересующее нас расстояние, соответственно, находится в нижнем правом углу таблицы. Обратите внимание, что такое вычисление как бы обратно определению (мы начинаем с конца, а не с начала).

Первый столбец и первая колонка таблицы заполняются по определению. Далее для каждого $i, j$:

$$
\text{D}(i, j) = \min \begin{cases}
\text{D}(i - 1, j) + 1 \\
\text{D}(i, j - 1) + 1 \\
\text{D}(i - 1, j - 1) + \begin{cases}
0 & \text{если } a[i] = b[j], \\
1 & \text{в противном случае } \\
\end{cases}
\end{cases}
$$

Отсюда видно, что на самом деле нам не нужно хранить всю матрицу &mdash; достаточно последних двух строк на каждом шаге. Тогда нам нужно завести первые две строки, в цикле вычислять каждую следующую и вернуть значение, лежащее в последней ячейке последней строки.

### Исправление опечаток (спеллчекинг)

Идея: если у нас есть словарь (список словоформ русского языка), мы можем попробовать определить, написано ли какое-то слово, введённое пользователем, с опечаткой или нет. Для этого нам нужно (1) определить, есть ли это слово в нашем словаре, и (2) если нет, то поискать в словаре слово, отличающееся от введённого не больше, чем на какое-то определённое небольшое расстояние Левенштейна (РЛ). Если нашлось, то предлагаем исправление; если не нашлось, делаем вывод, что это не опечатка, а просто неизвестное слово.

Подход в лоб: посчитать расстояние Левенштейна между введённым словом и всеми словами в словаре, взять то, у которого расстояние минимальное. Минус подхода: будет работать очень долго, т.к. расстояние Левенштейна &mdash; "дорогой" алгоритм, а в словаре может быть много слов.

Подход более умный (который надо реализовать):
1. Если введённое слово есть в словаре, то всё хорошо, дальше можно не идти.
2. Если нет, то для введённого слова сгенерируем все возможные русские слова/псевдослова, отстоящие от него на расстояние Левенштейна, равное 1. Для этого нужно провести все возможные элементарные операции: удаления (по одной удаляем из слова каждую букву), замены (по одной заменяем каждую букву в слове на каждую букву из русского алфавита), вставки (по одной вставляем каждую букву русского алфавита в каждую возможную позицию)

Тогда для слова "мама" все возможные слова с РЛ=1:

`all_edits = ["ама", "мма", "маа", "мам", "аама", "бама", ..., "мамю", "мамя", "амама", "бмама", ..., "маама", "мбама", ..., "мамаю", "мамая"]`

3. Посмотрим, какие слова из этого списка есть в словаре.

4. Если ни одного из этих слов нет в словаре, то тогда повторим пункт 2 для каждого слова из all_edits. Соединив результаты вместе, получим список слов, отстоящих от исходного на РЛ=2.

5. Повторим пункт 3.

6. Если и этих слов нет в словаре, делаем вывод, что опечатки нет, а просто слово неизвестно.

7. В противном случае возвращаем список полученных слов, ранжированный по их частотности.

В файле fontanka_freqs.txt содержится частотный список словоформ русского языка. Числа справа от слов &mdash; их частотность в логарифмическом представлении. Чем больше число (по значению, а не по модулю), тем более частотное слово.

Вопросы:  
1. В чём преимущество хранения частотности (т.е. вероятности встретить слово) в логарифмическом представлении?
2. Если нам нужно определить частотность сочетания двух слов (вероятность встретить два слова вместе), что нужно сделать? А в лог-представлении?

Полезная информация: время поиска в списке гораздо больше, чем в словаре или множестве.

In [ ]:
import time
import random

In [ ]:
a_set = {random.randint(1, 10 ** 9) for _ in range(10**7)}

In [ ]:
a_dict = {i: 0 for i in a_set}

In [ ]:
a_list = list(a_set)

In [ ]:
start_time = time.time()
for t in range(100):
    t in a_dict
print(time.time() - start_time)

In [ ]:
start_time = time.time()
for t in range(100):
    t in a_set
print(time.time() - start_time)

In [ ]:
start_time = time.time()
for t in range(100):
    t in a_list
print(time.time() - start_time)

### Модуль для расстояния Левенштейна

https://rapidfuzz.github.io/Levenshtein/

In [ ]:
!pip install levenshtein

In [ ]:
from Levenshtein import distance
distance("стала", "стол")

### Задание для самостоятельного выполнения

**Основное задание**: реализовать вычисление расстояния Левенштейна.

**Задание со звёздочкой**: у расстояния Левенштейна есть модификация &mdash; расстояние Дамерау&ndash;Левенштейна. При его вычислении в число минимальных операций входит ещё и перестановка соседних символов.

Чтобы его вычислить, нам нужно изменить алгоритм вычисления матрицы следующим образом:

$$
\text{D}(i, j) = \min \begin{cases}
\text{D}(i - 1, j) + 1 \\
\text{D}(i, j - 1) + 1 \\
\text{D}(i - 1, j - 1) + \begin{cases}
0 & \text{если } a[i] = b[j], \\
1 & \text{в противном случае } \\
\end{cases} \\
\text{D}(i - 2, j - 2) + \begin{cases}
0 & \text{если } a[i] = b[j], \\
1 & \text{в противном случае } \\
\end{cases} & \text{если } a[i] = b[j-1] \text{ и } a[i - 1] = b [j] \\
\end{cases}
$$

Теперь нам не хватает двух строк матрицы!

In [ ]:
def damerau_levenshtein_distance(a, b):
    ...

### Домашнее задание

Реализуйте алгоритм для исправления опечаток.